In [58]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [59]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [60]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
documents=SimpleDirectoryReader("data").load_data()

In [61]:
documents


[Document(id_='781547e7-d5e4-4847-9453-87e5a79f6469', embedding=None, metadata={'page_label': '1', 'file_name': '/Users/swapnildhamu/Developer/vin-ai/RAG/llamaindex_and_openAI/data/SKU Import _ Vinculum Knowledge Central.pdf', 'file_path': '/Users/swapnildhamu/Developer/vin-ai/RAG/llamaindex_and_openAI/data/SKU Import _ Vinculum Knowledge Central.pdf', 'file_type': 'application/pdf', 'file_size': 242159, 'creation_date': '2024-03-04', 'last_modified_date': '2024-03-03'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='03/03/2024, 12:06 SKU Import : Vinculum Knowledge Central\nhttps://vinculumhelpdesk.freshdesk.com/support/solutions/articles/9000213260-sku-import 1/3SKU Import\nModiﬁed on: Wed, 23 Mar, 2022 at 8:08 PM\nSKU Import\nIntroduction\nThe SKU im

In [62]:
index=VectorStoreIndex(documents, show_progress=True)

Generating embeddings: 100%|██████████| 10/10 [00:01<00:00,  6.04it/s]


In [63]:
index

In [64]:
query_engine=index.as_query_engine() 

In [65]:
query_engine

In [72]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor


retriever=VectorIndexRetriever(index=index,similarity_top_k=4)


postProcessor=SimilarityPostprocessor(similarity_cutoff=0.82)

query_engine=RetrieverQueryEngine(retriever=retriever,node_postprocessors=[postProcessor])

In [73]:
response=query_engine.query("i want to edit any sku field")

In [75]:
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(response,show_source=True)

Final Response: To edit any SKU field, you need to navigate to the SKU
Enquiry screen and click on the 'Blue' colored SKU code. This action
will open the SKU Create/Edit screen, where you can make the necessary
changes to the SKU details.
______________________________________________________________________
Source Node 1/1
Node ID: d6df5639-6533-4aa2-9a2a-7727766db2f1
Similarity: 0.8235599297663586
Text: 03/03/2024, 12:05 SKU Master : Vinculum Knowledge Central https:
//vinculumhelpdesk.freshdesk.com/support/solutions/articles/9000213659
-sku-master 7/7Shipping Charges This reflects the charges required for
shipping the SKU. Handling Charges This reflects the charges required
for handling the SKU. Def Source WHThis is used to define the
warehouse ...


In [69]:
print(response)

To edit any SKU field, you need to navigate to the SKU Enquiry screen and click on the 'Blue' colored SKU code. This action will open the SKU Create/Edit screen, where you can make the necessary changes to the SKU details.


## Stroing the index

In [70]:
import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage
)

PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

# either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query("What are SKU?")
print(response)

A Stock Keeping Unit or SKU is a distinct type of item that can be purchased, sold, or tracked with a unique alphanumeric code/scannable bar code printed on it or its packaging. This code contains all the attributes associated with that item-type that can distinguish it from other item-types. These attributes can be manufacturer name, material used, color, size, warranty, etc. The term SKU is used by stores, e-commerce vendors, warehouses, product fulfillment centers, and any other business in the inventory management field.


In [71]:
import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage
)

PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
    print("Created and stored index")
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)
    print("Loaded index from storage")

# either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query("What is SKU Import module?")
print(response)

Loaded index from storage
The SKU Import module is a feature in Vin eRetail that allows users to import SKUs in bulk using a CSV template file. It consists of two sections: Import and Download. The Import section enables users to import SKUs in bulk by selecting the Import Type, Attribute Set, and uploading the correct CSV file. The Download section allows users to search and download the uploaded CSV file of SKU imports based on various criteria.


In [ ]:
import shutil
import os
import pandas as pd
import logging
import time

log_file = 'IUCN_logfile.log'
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

directory = 'demo/'  # Path to the folder containing company folders
existing_excel_path = "d3.csv"  # Path to the file to be replicated every time

def filter_file(file_path, existing_csv_path):
    try:
        if file_path.endswith('.xlsx'):
            df_filtered = pd.read_excel(file_path)
        elif file_path.endswith('.csv'):
            df_filtered = pd.read_csv(file_path, encoding='latin1')
        else:
            print(f"Unsupported file format: {file_path}")
            return
        
        df_filtered = df_filtered[df_filtered['IUCN_Proximity'].str.lower() == 'yes']
        df_filtered = df_filtered.rename(columns={'Asset_ID': 'Asset Code', 'NAME': 'IUCN Species Name', 'Num_PAs': 'IUCN Intersection'})
        df_filtered['Buffer'] = 1
        df_filtered['IUCN Intersection']=100
        selected_columns = ['Asset Code', 'Buffer', 'IUCN Species Name', 'IUCN Intersection']
        df_filtered = df_filtered[selected_columns]
        
        existing_df = pd.read_csv(existing_csv_path, encoding='utf-8')
        
        combined_df = pd.concat([existing_df, df_filtered], ignore_index=True)
        
        combined_df.to_csv(existing_csv_path, index=False, encoding='utf-8')
        
        print(f"Filtered values from {file_path} inserted into {existing_csv_path}")
        logging.info(f"Filtered values from {file_path} inserted into {existing_csv_path}")
    except Exception as e:
        print(f"Error processing file {file_path}: {str(e)}")
        logging.error(f"Error processing file {file_path}: {str(e)}")

files_processed = 0
replicated_files_count = 0
replicated_csv_path = f"IUCN_group_{replicated_files_count}.csv"

shutil.copyfile(existing_excel_path, replicated_csv_path)
print("CSV file replicated successfully!")
logging.info("CSV file replicated successfully!")

start_time = time.time()
for root, dirs, files in os.walk(directory):
    for file in files:
        if file in ['IUCN.xlsx', 'IUCN.csv']:
            file_path = os.path.join(root, file)
            filter_file(file_path, replicated_csv_path)
            files_processed += 1
            group = 3
            if files_processed % group == 0:
                replicated_files_count += 1
                replicated_csv_path = f"IUCN_group_{replicated_files_count}.csv"
                shutil.copyfile(existing_excel_path, replicated_csv_path)
                print("CSV file replicated successfully!")
                logging.info("CSV file replicated successfully!")
end_time = time.time()
execution_time = end_time - start_time
print(f"Total execution time: {execution_time} seconds")